# Lightweight python components

Lightweight python components do not require you to build a new container image for every code change.
They're intended to use for fast iteration in notebook environment.

#### Building a lightweight python component
To build a component just define a stand-alone python function and then call kfp.components.func_to_container_op(func) to convert it to a component that can be used in a pipeline.

There are several requirements for the function:
* The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
* The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package.)
* If the function operates on numbers, the parameters need to have type hints. Supported types are ```[int, float, bool]```. Everything else is passed as string.
* To build a component with multiple output values, use the typing.NamedTuple type hint syntax: ```NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])```

In [8]:
EXPERIMENT_NAME = 'lightweight python components'
KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.7/kfp.tar.gz'

In [9]:
# Install the SDK
!pip3 install $KFP_PACKAGE --upgrade


  Using cached https://storage.googleapis.com/ml-pipeline/release/0.1.7/kfp.tar.gz
  Running setup.py bdist_wheel for kfp ... done
  Stored in directory: /private/var/folders/4t/5r35f1f15nb42k1pgy6myh4w0000gp/T/pip-ephem-wheel-cache-74fdts5v/wheels/d4/2c/9a/f56f8f1e4de577e459981a810ac6228805ceec3b19c56527a7
Successfully built kfp
  Found existing installation: kfp 0.1
    Uninstalling kfp-0.1:
      Successfully uninstalled kfp-0.1


In [10]:
import kfp.components as comp

In [19]:
from kubernetes import client, config

In [33]:
config.load_kube_config(config_file='/Users/adselvar/.kube/config')
v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s" %
          (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

/usr/local/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Listing pods with their IPs:
10.40.7.9	default	wistful-giraffe-0
10.40.8.5	default	wistful-giraffe-1
10.40.4.12	default	wistful-giraffe-2
10.40.6.13	default	wistful-giraffe-3
10.40.4.4	istio-system	istio-citadel-84fb7985bf-shnt9
10.40.4.5	istio-system	istio-egressgateway-bd9fb967d-fwqdk
10.40.4.6	istio-system	istio-galley-655c4f9ccd-zl9ll
10.40.5.4	istio-system	istio-ingressgateway-688865c5f7-mggpv
10.40.6.4	istio-system	istio-pilot-6cd69dc444-n7kd4
10.40.6.3	istio-system	istio-policy-6b9f4697d-bgl74
10.40.7.4	istio-system	istio-sidecar-injector-8975849b4-n8b75
10.40.5.6	istio-system	istio-statsd-prom-bridge-7f44bb5ddb-d5qf8
10.40.5.9	istio-system	istio-telemetry-6b5579595f-n5glh
10.40.5.8	kube-system	event-exporter-v0.2.3-54f94754f4-5r4mx
10.40.6.5	kube-system	fluentd-gcp-scaler-6d7bbc67c5-j4wfd
10.40.4.2	kube-system	fluentd-gcp-v3.1.0-qrfhx
10.40.7.3	kube-system	fluentd-gcp-v3.1.0-qtv74
10.40.8.2	kube-system	fluentd-gcp-v3.1.0-sbq5s
10.40.5.3	kube-system	fluentd-gcp-v3.1.0-wmbsv
10.4

Simple function that just add two numbers:

In [34]:
#Define a Python function
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

Convert the function to a pipeline operation

In [35]:
add_op = comp.func_to_container_op(add)

A bit more advanced function which demonstrates how to use imports, helper functions and produce multiple outputs.

In [36]:
#Advanced function
#Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple
def my_divmod(dividend: float, divisor:float, output_dir:str = './') -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    '''Divides two numbers and calculate  the quotient and remainder'''
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }
    with open(output_dir + 'mlpipeline-ui-metadata.json', 'w') as f:
      json.dump(metadata, f)

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'quotient',
          'numberValue':  float(quotient),
        },{
          'name': 'remainder',
          'numberValue':  float(remainder),
        }]}

    with file_io.FileIO(output_dir + 'mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

Test running the python function directly

In [37]:
my_divmod(100, 7)

ModuleNotFoundError: No module named 'tensorflow'

#### Convert the function to a pipeline operation

You can specify an alternative base container image (the image needs to have Python 3.5+ installed).

In [38]:
divmod_op = comp.func_to_container_op(my_divmod, base_image='tensorflow/tensorflow:1.11.0-py3')

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator and the parameters must have default values of type `dsl.PipelineParam`

In [39]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='a',
   b='7',
   c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    divmod_task = divmod_op(add_task.output, b, '/')

    #For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax
    result_task = add_op(divmod_task.outputs['quotient'], c)

#### Compile the pipeline

In [40]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.tar.gz'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [41]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#vvvvvvvvv This link leads to the run information page. (Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working)

SyntaxError: invalid syntax (experiment_service_api.py, line 127)